# 基于长词黑名单去除高频广告

In [1]:
import numpy as np

In [2]:
#相邻3个长词，视为异常
thr = 3
black_list = set()
for word_len in range(5,8):
    history_words = [""] * (thr - 1)
    history_value = [0] * (thr - 1)
    cnt = 0
    with open('vocabs/vocab_941993.txt') as f:
        for pair in f:
            if pair[0] != '\t' and len(pair.split('\t')[0])==word_len:
                word,value = pair.split('\t')
                if not "" in history_words and history_value[0]<10000 and len(set(history_value)) == 1:
                    black_list |= set(history_words)
                history_words[cnt]=word
                history_value[cnt]=int(value)//10
            else:
                history_words[cnt] = ""
            cnt = (cnt + 1) % (thr - 1)

In [3]:
len(black_list)

724

In [4]:
#随机查看广告词
np.array(list(black_list))[np.random.randint(0,len(black_list),60)]

array(['使用两个周', '化为发展目标的', '药名词术语成果', '经常做水果面贴', '纳四方医学精英', '道工艺炼制成霜',
       '拉紧致眼部轮廓', '等面目神经疾病', '疗程时间比较长', '乳腺脂肪坏死等', '药店基本都有售', '默认的收益分配',
       '双重认证的专业', '监督管理核设施', '迅速提升精力体', '死亡的主要形式', '大笑或者大喊时', '自然科学事业费',
       '精索静脉曲张吗', '各种毒素和垃圾', '我们在收到短信', '十大惨不忍睹的', '吃过其他减肥药', '代谢减缓等问题',
       '绿色生态祛疤', '塑形以及美胸三', '理事会的职权是', '链接受损皮肤层', '偷情中的傻女', '目前市面上的去',
       '欧雅泊思官网上', '监督管理核设施', '与肌肤完全相容', '可能引发尿失禁', '在核准的业务范', '乳房的症状同步',
       '面部医疗费用的', '舒适的住院病区', '够快速分解皮肤', '由理事会或常务', '指由于乳房组织', '行政诉讼的应诉',
       '有什么技术革新', '这些患者普遍', '铺设了地毯的地', '处理方法是', '渗透因子的包裹', '调查情况及结果',
       '找一个不可', '经常做水果面贴', '姓节省医疗开支', '何找到更多有关', '凸起的青筋消失', '乾正康面瘫静电',
       '以复发率非常高', '建议按周期使用', '而且还能保证患', '的吸金能力', '体温下自行变软', '东西就发生射'],
      dtype='<U7')

In [5]:
import ahocorasick as ah
aca= ah.Automaton()
for word in black_list:
    #构造自动机时对词频进行过滤
    aca.add_word(word.encode(),None)
aca.make_automaton()

In [6]:
def isad(text):
    cnt = 0
    for end_index,_ in aca.iter(text.encode()):
        cnt += 1
        if cnt >= 3:
            return True
    return False

In [7]:
with open('high_ff_final_random.txt','r') as f_in:
    with open('high_ff_final2_random.txt','w') as f_out:
        for line in f_in:
            if not isad(line):
                print(line,end="",file=f_out)